In [1]:
# import tensorflow as tf
# import os

# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

# strategy = tf.distribute.TPUStrategy(resolver)

In [2]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [3]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# import numpy as np

# # 한글 지원 폰트
# plt.rc('font', family='NanumBarunGothic')

# # 마이너스 부호 
# plt.rcParams['axes.unicode_minus'] = False

In [4]:
! pip install transformers
! pip install datasets

In [5]:
# 시드 고정
import torch
import numpy as np
import random
import os

def my_seed_everywhere(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # numpy
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    # pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

my_seed = 42
my_seed_everywhere(my_seed)

import tensorflow as tf
import numpy as np
import random
import os

def my_seed_everywhere(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # np
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    tf.random.set_seed(seed) # tensorflow

my_seed = 42
my_seed_everywhere(my_seed)

In [6]:
# # Mecab 설치
# !apt-get update
# !apt-get install g++ openjdk-8-jdk 
# !pip3 install konlpy JPype1-py3
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tqdm
from transformers import Trainer, TrainingArguments
import re
import pyarrow as pa
from datasets import Dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from sklearn.model_selection import train_test_split
from datasets import load_metric

## 데이터 불러오기

In [8]:
dir_path = '/aiffel/aiffel/AIFFELTHON/data/test_1125/'
file_name = 'train_bt_en_fr'
data = pd.read_csv(dir_path+file_name+'.csv')
data.head(10)

,class,non_label_sentence
0,직장 내 괴롭힘 대화,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...
1,기타 괴롭힘 대화,야 너 앞니 왜 그렇게 튀어나왔냐\n태어날 때부터 그랬어 물어보지 마\n아 그럼 태...
2,갈취 대화,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n미안 나 오늘은 진짜 돈이 ...
3,갈취 대화,너 저번에 술 먹은 날 기억해\n아니 왜\n야 300만 원만 가져와\n뭔 소리야 ...
4,협박 대화,너 또 내말 무시하냐\n이 새끼 널 좆으로 보나 본데\n아냐 진짜 시간이 없었어 미...
5,갈취 대화,야 돼지 오늘도 돈 내놔\n오늘은 진짜 없어\n아새끼 오늘 진짜 돈 안 들고 왔네\...
6,기타 괴롭힘 대화,오 신발 샀냐\n응 전에 신던 게 다 헤져서 부모님이 사주셨어\n너네 집 꽤 사...
7,기타 괴롭힘 대화,영하 학생 이리 나와요\n네 선생님\n선생님이 왜 부른 줄 알아\n잘 모르겠습니다 ...
8,직장 내 괴롭힘 대화,씨 오늘 뭐 해요\n아 저 약속이 있어서요\n누구랑요 남자친구\n아 네\n에이 ...
9,갈취 대화,야 죽기 싫으면 돈 내놔\n죄송합니다 여기\n뭔 이거 밖 없어 뒤져서 나오면 죽...


## 데이터 전처리

In [9]:
labels = {'협박 대화':0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3, '일반 대화':4}

num_labels = len(labels)

data['label'] = data['class'].apply(lambda x: labels[x])
data.head(10)

,class,non_label_sentence,label
0,직장 내 괴롭힘 대화,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,2
1,기타 괴롭힘 대화,야 너 앞니 왜 그렇게 튀어나왔냐\n태어날 때부터 그랬어 물어보지 마\n아 그럼 태...,3
2,갈취 대화,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n미안 나 오늘은 진짜 돈이 ...,1
3,갈취 대화,너 저번에 술 먹은 날 기억해\n아니 왜\n야 300만 원만 가져와\n뭔 소리야 ...,1
4,협박 대화,너 또 내말 무시하냐\n이 새끼 널 좆으로 보나 본데\n아냐 진짜 시간이 없었어 미...,0
5,갈취 대화,야 돼지 오늘도 돈 내놔\n오늘은 진짜 없어\n아새끼 오늘 진짜 돈 안 들고 왔네\...,1
6,기타 괴롭힘 대화,오 신발 샀냐\n응 전에 신던 게 다 헤져서 부모님이 사주셨어\n너네 집 꽤 사...,3
7,기타 괴롭힘 대화,영하 학생 이리 나와요\n네 선생님\n선생님이 왜 부른 줄 알아\n잘 모르겠습니다 ...,3
8,직장 내 괴롭힘 대화,씨 오늘 뭐 해요\n아 저 약속이 있어서요\n누구랑요 남자친구\n아 네\n에이 ...,2
9,갈취 대화,야 죽기 싫으면 돈 내놔\n죄송합니다 여기\n뭔 이거 밖 없어 뒤져서 나오면 죽...,1


In [10]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report

# # 대화와 라벨 분류
# train = data['conversation'].tolist()
# target = data['label'].tolist()

# # train, val 분리 (20%)
# train_x, val_x, train_y, val_y = train_test_split(train, target, random_state=27, test_size=0.2)

In [11]:
model_path = "beomi/KcELECTRA-base"

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels = 5)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.ou

## Tokenizer 및 Mdoel 불러오기

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import ElectraTokenizer, ElectraForSequenceClassification

HUGGINGFACE_MODEL_PATH = "beomi/KcELECTRA-base"

tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=5)
#model.num_labels=5
#model = ElectraForSequenceClassification.from_pretrained("beomi/KcELECTRA-base", num_labels=num_labels)


# train_tensor = tokenizer(train_x, truncation=True, padding=True)
# val_tensor = tokenizer(val_x, truncation=True, padding=True)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.ou

In [13]:
# # trainset-set
# train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(train_tensor),
#     train_y
# ))

# # validation-set
# val_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(val_tensor),
#     val_y
# ))

## 학습을 위한 데이터셋 생성

In [14]:
import pyarrow as pa
from datasets import Dataset

data = Dataset(pa.Table.from_pandas(data))

In [15]:
print(data.data)

InMemoryTable
class: string
non_label_sentence: string
label: int64
----
class: [["직장 내 괴롭힘 대화","기타 괴롭힘 대화","갈취 대화","갈취 대화","협박 대화","갈취 대화","기타 괴롭힘 대화","기타 괴롭힘 대화","직장 내 괴롭힘 대화","갈취 대화",...,"일반 대화","일반 대화","일반 대화","일반 대화","일반 대화","일반 대화","일반 대화","일반 대화","일반 대화","일반 대화"]]
non_label_sentence: [["길동 씨 이번에 이것 좀 처리해요
이거 제가 한 게 아닌데요
팀에서 내가 네가 가 어딨어
그래도 이건 제가 쓴 게 아닌데
이번 인사고과기간인거 알지
네  아니 이건 제가 덤터기 쓸 텐데요
이번 거만 해주면 길동 씨 데리고 갈게
네
내가 길동 씨 계약만료하자고 할까
네 제가 처리할게요","야 너 앞니 왜 그렇게 튀어나왔냐
태어날 때부터 그랬어 물어보지 마
아 그럼 태어날 때부터 못 생기 거네
괴롭히지 마 제발
아니 너는 내가 말만 하면 괴롭혔다고 한다
혼자 있게 해줘
제 혼자 있는 거 같아서 말 걸어줬더니만 나쁜 사람 만드네 그래놓고 왕따시켰다고 말하지나 마 진짜 성격이 왜 그따위야  가정교육 잘못 받았냐
부모님은 건드리지 마
어쭈 발끈한다 지도 자식이라 이거지
얘 왜 우냐
아니 몰라 제 혼자 갑자기 화내
제발 혼자 있게 해줘 내버려 둬
이거 왜 또 제 혼자 감성적이냐  애니 그만 보라 했지
완전 얘 코미디언 아니냐","원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼
미안 나 오늘은 진짜 돈이 없어
나고 진짜 돈이 없어서 그래  너 비상금은
그 돈은 정말 쓰면 안 되는 돈이야
그냥 달라는 게 아니라 갚을 거야
비상금은 정말 안돼
왜 이렇게 쪼잔하게 굴어
그렇지만
내가 손 좀 봐줘야 정신 차려
마지막이야","너 저번에 술 먹은 날 기억해
아니 왜
야 300만 원만 가져와
뭔 소리야  갑자기 왜 이래
너 술 먹인 날 모텔방 가서 섹스비디오 찍

In [16]:
def transform(data):
  return tokenizer(
      data['non_label_sentence'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )

encoded_data = data.map(transform, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

In [17]:
train_dataset, validation_dataset= encoded_data.train_test_split(test_size=0.1).values()

In [18]:
train_dataset

Dataset({
    features: ['attention_mask', 'class', 'input_ids', 'label', 'non_label_sentence', 'token_type_ids'],
    num_rows: 12947
})

In [19]:
metric = load_metric("f1")

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels, average="micro")

output_dir = '/content/drive/MyDrive/공부/AIFFELTHON/etc'

training_arguments = TrainingArguments(
    output_dir, 
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, 
    per_device_train_batch_size = 8, 
    per_device_eval_batch_size = 8, 
    num_train_epochs = 3,
    weight_decay = 0.01
)

In [20]:
trainer = Trainer(
    model=model,                           
    args=training_arguments,                  
    train_dataset=train_dataset,    
    eval_dataset=validation_dataset,      
    compute_metrics=compute_metrics
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: class, non_label_sentence.
***** Running training *****
  Num examples = 12947
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4857


Epoch,Training Loss,Validation Loss,F1
1,0.332200,0.423946,0.899236
2,0.159200,0.325465,0.939541
3,0.089200,0.264037,0.952745


Saving model checkpoint to /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-500
Configuration saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1000
Configuration saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1500
Configuration saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1500/config.json
Model weights saved in /content/drive/MyDrive/공부/AIFFELTHON/etc/checkpoint-1500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: clas

TrainOutput(global_step=4857, training_loss=0.2620278441515729, metrics={'train_runtime': 4122.1259, 'train_samples_per_second': 9.423, 'train_steps_per_second': 1.178, 'total_flos': 1.0219771771444224e+16, 'train_loss': 0.2620278441515729, 'epoch': 3.0})

In [21]:
# with strategy.scope():
#     model = TFBertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=num_labels, from_pt=True)
#     optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
#     model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['f1'])
#model = TFBertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels = num_labels)

In [22]:
# trainer = Trainer(
#     model=model,                           
#     args=training_arguments,                  
#     train_dataset=train_dataset,    
#     eval_dataset=val_dataset,      
#     compute_metrics=compute_metrics
# )

In [23]:
# with strategy.scope():
#     trainer.train()

In [24]:
# history = model.fit(
#     train_dataset.shuffle(len(train_dataset)).batch(8), epochs=3, batch_size=8,
#     validation_data=val_dataset.shuffle(len(val_dataset)).batch(8)
# )

In [25]:
# id2labels = model.config.id2label
# model.config.id2label = {id : label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0]  for id, label in id2labels.items()}

# label2ids = model.config.label2id
# model.config.label2id = {label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0] : id   for id, label in id2labels.items()}

In [26]:
# import os

# DIR_PATH = '/content/drive/MyDrive/공부/AIFFELTHON/model'
# MODEL_NAME = 'fine-tuned-KcELECTRA-base'
# MODEL_SAVE_PATH = os.path.join(DIR_PATH, MODEL_NAME) # change this to your preferred location

# if os.path.exists(MODEL_SAVE_PATH):
#     print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
# else:
#     os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
#     print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# # save tokenizer, model
# model.save_pretrained(MODEL_SAVE_PATH)
# tokenizer.save_pretrained(MODEL_SAVE_PATH)

In [27]:
# from transformers import TextClassificationPipeline, BertTokenizer

# # Load Fine-tuning model
# loaded_tokenizer = BertTokenizer.from_pretrained(MODEL_SAVE_PATH)
# loaded_model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

In [28]:
# from transformers import TextClassificationPipeline

# text_classifier = TextClassificationPipeline(
#     tokenizer=tokenizer, 
#     model=model, 
#     framework='tf'
# )

# text_classifier(test_x[0])

In [29]:
test_file_path = '/aiffel/aiffel/AIFFELTHON/data/test.json'
with open(test_file_path, mode='rt', encoding='utf-8') as f:
    test_dataset = pd.read_json(f)

test_data = test_dataset.transpose()
test_data

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
...,...
t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...
t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...
t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...
t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...


In [30]:
def remove_punctuation(x):
  x = x.strip()
  x = re.sub("[^가-힣0-9]+", " ", x)
  x = re.sub("[ ]+", " ", x)
  x = x.strip()
  return x

test_data["text"] = test_data['text'].apply(lambda x : remove_punctuation(x))
test_data.head()

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나 그럼 영지씨가 가는건 어때 네 제가요 그렇지 2...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요 제대로 좀 하지 네 똑바로 좀 하지 행실...
t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
t_004,아무튼 앞으로 니가 내 와이파이야 응 와이파이 온 켰어 반말 주인님이라고도 말해야지...


In [31]:
test_X = Dataset(pa.Table.from_pandas(test_data))
#test_X = test_data['text']

In [32]:
def test_transform(data):
  return tokenizer(
      data['text'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )

encoded_test_data = test_X.map(test_transform, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [33]:
prediction = trainer.predict(encoded_test_data)

The following columns in the test set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: text, __index_level_0__.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


In [34]:
len(prediction.predictions)
pred_label = []

for i in range(len(prediction.predictions)):
    pred_label.append((str)(np.argmax(prediction.predictions[i])).zfill(2))
    
pred_label[:10]

['01', '02', '02', '04', '03', '00', '00', '01', '04', '01']

In [35]:
submission = test_data.assign(CLASS=pred_label)
submission = submission.rename(columns={'CLASS':'class'})

submission.drop(['text'], axis=1, inplace=True)
submission = submission.transpose()
submission

,t_000,t_001,t_002,t_003,t_004,t_005,t_006,t_007,t_008,t_009,...,t_490,t_491,t_492,t_493,t_494,t_495,t_496,t_497,t_498,t_499
class,01,02,02,04,03,00,00,01,04,01,...,03,04,00,01,01,02,02,01,02,00


In [36]:
import json

model_name = 'KcELECTRA'

submission_file_path = '/aiffel/aiffel/AIFFELTHON/submission/test_1125/' + 'submission_' + model_name + file_name[5:] + '.json'
result = submission.to_json(submission_file_path)

with open(submission_file_path) as f:
    parsed = json.load(f)

with open(submission_file_path, 'w') as f:
    json.dump(parsed, f, indent=4)